In [1]:
import cv2
import argparse
import imutils
import time
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from PIL import Image
from imutils.video import VideoStream

In [2]:
MODEL_DIR = '../../model/'
IMG_HEIGHT = 256
IMG_WIDTH = 256
CLASS_NAMES = ['correctly-masked', 'not-masked', 'uncorrectly-masked']

In [3]:
model = keras.models.load_model(MODEL_DIR)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 256, 256, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 64, 64, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 131072)            0

In [4]:
def get_face(img, x, y, w, h) -> Image.Image: 
    crop_img = img[y:y+h, x:x+w]
    cv2.imshow("cropped", crop_img)
    # Convert image from GBR to RGB
    rgb_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)
    pil_img = Image.fromarray(rgb_img)
    newsize = (IMG_WIDTH, IMG_HEIGHT)
    resized_img = pil_img.resize(newsize)
    return resized_img

def predict(resized_img):
    img_array = keras.preprocessing.image.img_to_array(resized_img)
    img_array = tf.expand_dims(img_array, 0)
    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    prediction = CLASS_NAMES[np.argmax(score)]
    confidence = 100 * np.max(score)
    return prediction, confidence
    
def get_color(prediction) -> tuple:
    green = (36,255,12)
    if prediction == CLASS_NAMES[0]:
        color = green
    elif prediction == CLASS_NAMES[1]:
        color = (0, 255, 0)
    else:
        color = (0, 0, 255)

In [5]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
print("[INFO] starting video stream ...")
cap = cv2.VideoCapture(0)
_, img = cap.read()
if (cap.isOpened() == False):
    print("[ERROR] Unable to read camera feed ...")

while True:
    _, img = cap.read()
    # Convert to grayscale (haarcascade works with grayscale)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    for (x, y, w, h) in faces:
        resized_img = get_face(img, x, y, w, h)
        prediction, confidence= predict(resized_img)
        color = get_color(prediction)
        print(f'{prediction}: {confidence}')
        cv2.rectangle(img, (x, y), (x+w, y+h), (36,255,12), 2)
        cv2.putText(img, str(prediction), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
        
    cv2.imshow('img', img)
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
exit()

[INFO] starting video stream ...
not-masked: 95.63136100769043
not-masked: 93.21862459182739
not-masked: 92.62665510177612
not-masked: 91.81388020515442
not-masked: 97.84598350524902
not-masked: 94.45226788520813
not-masked: 93.56792569160461
not-masked: 93.55699419975281
not-masked: 94.50055360794067
not-masked: 97.71719574928284
not-masked: 97.64727354049683
not-masked: 98.06468486785889
not-masked: 99.10477995872498
not-masked: 98.93608689308167
not-masked: 99.08711314201355
not-masked: 98.5969066619873
not-masked: 98.846834897995
not-masked: 98.6871600151062
not-masked: 98.53945970535278
not-masked: 98.71121048927307
not-masked: 96.5469241142273
not-masked: 97.94514775276184
not-masked: 94.91588473320007
not-masked: 93.55576038360596
not-masked: 91.38045907020569
not-masked: 92.65714287757874
not-masked: 94.1960334777832
not-masked: 93.63782405853271
not-masked: 99.70223307609558
not-masked: 99.10900592803955
not-masked: 97.0333993434906
not-masked: 96.83389663696289
not-masked: 95